In [12]:
import os
import numpy as np
import pandas as pd
from time import time
from utils import custom_preprocessing, dependencies, aspect_tools

data_dir = "/home/stavros/DATA/AirbnbReviews"
#data_dir = "D:/"

In [2]:
area = "athens"
area_dir = os.path.join(data_dir, area)

listings = pd.read_csv(os.path.join(area_dir, "listings.csv.gz"))

/home/stavros/PythonEvironments/neuralcoref-env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
reviews = pd.read_pickle(os.path.join(area_dir, "reviews_with_aspects_210000samples.pkl"))
print(reviews.shape)
aspect_reviews = reviews[pd.notnull(reviews["aspects"])]
print(aspect_reviews.shape)
aspect_reviews = aspect_reviews[aspect_reviews["aspects"].map(lambda x: len(x) > 1)]
print(aspect_reviews.shape)

(379118, 8)
(210000, 8)
(172455, 8)


In [10]:
for c in listings.columns:
    print(c)

id
listing_url
scrape_id
last_scraped
name
summary
space
description
experiences_offered
neighborhood_overview
notes
transit
access
interaction
house_rules
thumbnail_url
medium_url
picture_url
xl_picture_url
host_id
host_url
host_name
host_since
host_location
host_about
host_response_time
host_response_rate
host_acceptance_rate
host_is_superhost
host_thumbnail_url
host_picture_url
host_neighbourhood
host_listings_count
host_total_listings_count
host_verifications
host_has_profile_pic
host_identity_verified
street
neighbourhood
neighbourhood_cleansed
neighbourhood_group_cleansed
city
state
zipcode
market
smart_location
country_code
country
latitude
longitude
is_location_exact
property_type
room_type
accommodates
bathrooms
bedrooms
beds
bed_type
amenities
square_feet
price
weekly_price
monthly_price
security_deposit
cleaning_fee
guests_included
extra_people
minimum_nights
maximum_nights
minimum_minimum_nights
maximum_minimum_nights
minimum_maximum_nights
maximum_maximum_nights
minimum_ni

In [11]:
import gensim
google_vec_file = os.path.join(data_dir, "GoogleNews-vectors-negative300.bin.gz")
word2vec = gensim.models.KeyedVectors.load_word2vec_format(google_vec_file, binary=True, limit=200000)
word2vec

In [13]:
# Ignores sentiment
all_aspects = aspect_tools.collect_aspects(aspect_reviews)
print(len(all_aspects))

12107


In [14]:
all_aspects.most_common(10)

[('host', 67533),
 ('location', 55904),
 ('apartment', 55245),
 ('place', 49745),
 ('stay', 20838),
 ('you', 16957),
 ('view', 15652),
 ('communication', 10099),
 ('area', 9372),
 ('time', 9343)]

In [15]:
aspect_matrix = aspect_tools.DistanceMatrix.calculate(word2vec, all_aspects)

Calculating matrix with 3745 words.


In [17]:
merged_aspects = aspect_matrix.merge_words(cut_off=0.3)

In [33]:
import plotly.graph_objects as go
bar_plot_word_index0 = 0
bar_plot_word_index1 = 20

bar_plot_words = merged_aspects.most_common()[bar_plot_word_index0: bar_plot_word_index1]
bar_plot_words = [word for word, _ in bar_plot_words]
bar_plot_counts = np.array([merged_aspects[word] for word in bar_plot_words]) * 100.0 / len(aspect_reviews)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=bar_plot_words,
    x=bar_plot_counts,
    orientation="h",
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode="group", 
                  width=900,
                  height=700,
                  font_size=16, 
                  xaxis_title="Aspect occurence per review (%)",
                  bargap=0.15, # gap between bars of adjacent location coordinates.
                  bargroupgap=0.1, # gap between bars of the same location coordinate.
                  legend=dict(x=0.8, y=1.0,bgcolor='rgba(255, 255, 255, 0)', 
                              bordercolor='rgba(255, 255, 255, 0)', font_size=20)
                  )
fig.show()

In [32]:
import plotly.graph_objects as go
bar_plot_word_index0 = 20
bar_plot_word_index1 = 50

bar_plot_words = merged_aspects.most_common()[bar_plot_word_index0: bar_plot_word_index1]
bar_plot_words = [word for word, _ in bar_plot_words]
bar_plot_counts = np.array([merged_aspects[word] for word in bar_plot_words]) * 100.0 / len(aspect_reviews)

fig = go.Figure()
fig.add_trace(go.Bar(
    y=bar_plot_words,
    x=bar_plot_counts,
    orientation="h",
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode="group", 
                  width=900,
                  height=1000,
                  font_size=16, 
                  xaxis_title="Aspect occurence per review (%)",
                  bargap=0.2, # gap between bars of adjacent location coordinates.
                  bargroupgap=0.1, # gap between bars of the same location coordinate.
                  legend=dict(x=0.8, y=1.0,bgcolor='rgba(255, 255, 255, 0)', 
                              bordercolor='rgba(255, 255, 255, 0)', font_size=20)
                  )
fig.show()

Explore dates

In [65]:
from datetime import date
def todate(x):
    year, month, day = tuple(int(a) for a in x.split("-"))
    return date(year, month, day)

aspect_reviews["num_date"] = aspect_reviews["date"].map(todate)
aspect_reviews["year"] = aspect_reviews["date"].map(lambda x: int(x.split("-")[0]))
aspect_reviews["month"] = aspect_reviews["date"].map(lambda x: int(x.split("-")[1]))
aspect_reviews["day"] = aspect_reviews["date"].map(lambda x: int(x.split("-")[2]))

In [81]:
months = [""]
month_counts = np.array([(aspect_reviews["month"] == m).sum() for m in months])

fig = go.Figure(data=go.Scatter(x=months, y=month_counts, marker_size=8))
fig.update_layout(xaxis_title="Month", yaxis_title="Number or reviews")
fig.show()

In [83]:
years = np.arange(aspect_reviews["year"].min(), aspect_reviews["year"].max())
year_counts = np.array([(aspect_reviews["year"] == m).sum() for m in years])

fig = go.Figure(data=go.Scatter(x=years, y=year_counts, marker_size=8))
fig.update_layout(xaxis_title="Year", yaxis_title="Number or reviews")
fig.show()

In [85]:
cut_off = 0.3
word_replacement_map = {}
for i, word in enumerate(aspect_matrix.words):
    if word not in word_replacement_map:
        word_replacement_map[word] = word
        for j in np.where(aspect_matrix.matrix[i] < cut_off)[0]:
            if aspect_matrix.words[j] not in word_replacement_map:
                word_replacement_map[aspect_matrix.words[j]] = word
                
print(len(word_replacement_map))

3745


In [86]:
replace_words = lambda x: {word_replacement_map[w]: v for w, v in x.items() if w in word_replacement_map}
aspect_reviews["merged_aspects"] = aspect_reviews["aspects"].map(replace_words)

In [89]:
import collections
def collect_aspects(data_column: pd.Series) -> collections.Counter:
  """Collects all the aspects from a DataFrame.

  The DataFrame must have an `aspects` column that contains sets.
  """
  all_aspects = collections.Counter()
  for aspects in data_column:
    for phrase in aspects:
      for word in phrase.split(" "):
        all_aspects[word] += 1
  return all_aspects


month_aspects = [collect_aspects(aspect_reviews[aspect_reviews["month"] == m]["merged_aspects"])
                 for m in range(1, 13)]

In [90]:
for m in month_aspects:
    print(len(m))

1391
1331
1604
1879
2072
2178
2218
2098
2112
1880
1526
1317


In [113]:
winter_aspects = collections.Counter()
winter_months = [1, 2, 3, 4, 11, 12]
for m in winter_months:
    winter_aspects += month_aspects[m - 1]
# Normalize
norm = (aspect_reviews["month"].map(lambda x: x in winter_months)).sum()
print("Winter reviews:", norm)
winter_aspects = collections.Counter({k: v * 100.0 / norm for k, v in winter_aspects.items()})

summer_aspects = collections.Counter()
summer_months = [5, 6, 7, 8, 9, 10]
for m in summer_months:
    summer_aspects += month_aspects[m - 1]
# Normalize
norm = (aspect_reviews["month"].map(lambda x: x in summer_months)).sum()
print("Summer reviews:", norm)
summer_aspects = collections.Counter({k: v * 100.0 / norm for k, v in summer_aspects.items()})

Winter reviews: 48492
Summer reviews: 123963


In [116]:
bar_plot_word_index0 = 50
bar_plot_word_index1 = 70

collective_aspects = winter_aspects + summer_aspects
bar_plot_words = collective_aspects.most_common()[bar_plot_word_index0: bar_plot_word_index1]
bar_plot_words = [word for word, _ in bar_plot_words]
bar_plot_counts_winter = [winter_aspects[word] for word in bar_plot_words]
bar_plot_counts_summer = [summer_aspects[word] for word in bar_plot_words]

fig = go.Figure()
fig.add_trace(go.Bar(
    y=bar_plot_words,
    x=bar_plot_counts_winter,
    name="Winter",
    orientation="h",
))
fig.add_trace(go.Bar(
    y=bar_plot_words,
    x=bar_plot_counts_summer,
    name="Summer",
    orientation="h"
))

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(barmode="group", 
                  width=900,
                  height=700,
                  font_size=16, 
                  xaxis_title="Aspect occurence per review (%)",
                  bargap=0.15, # gap between bars of adjacent location coordinates.
                  bargroupgap=0.1, # gap between bars of the same location coordinate.
                  legend=dict(x=0.8, y=1.0,bgcolor='rgba(255, 255, 255, 0)', 
                              bordercolor='rgba(255, 255, 255, 0)', font_size=20)
                  )
fig.show()

In [119]:
winter_summer_diff = collections.Counter({k: np.abs(winter_aspects[k] - summer_aspects[k])
                                          for k, v in collective_aspects.items() if v > 1.0})
print(len(winter_summer_diff))

104


In [120]:
winter_summer_diff.most_common()

[('house', 1.044919231483611),
 ('everything', 0.8843899053361834),
 ('restaurants', 0.6978457001132421),
 ('apartment', 0.6652783174441588),
 ('you', 0.6484185278177375),
 ('time', 0.6110563564457188),
 ('location', 0.5794928475706449),
 ('view', 0.5781879364052482),
 ('host', 0.5521544421209228),
 ('problem', 0.5447207853726455),
 ('water', 0.5417776859254466),
 ('recommendations', 0.5350614882705127),
 ('balcony', 0.5081353123777665),
 ('it', 0.4776184972676356),
 ('stay', 0.43850639645424394),
 ('hospitality', 0.4330852783396826),
 ('bed', 0.4106547003273482),
 ('person', 0.3891343211842735),
 ('flat', 0.32809764665372443),
 ('check', 0.3061552063286488),
 ('communication', 0.2958006586262494),
 ('spot', 0.2786692765967962),
 ('space', 0.2741244706845225),
 ('choice', 0.24897592246609213),
 ('views', 0.24216624951331234),
 ('decoration', 0.23826369325826585),
 ('kitchen', 0.23029269079086334),
 ('cozy', 0.22930498677608502),
 ('beds', 0.22034792721586327),
 ('walk', 0.2175950722082